# Spark Block Matrix Tranform experiment

terra/deseq/bin/createSalmonNumReadsTransposeMatrix.sh create a matrix with the following form

```
Name,ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|,ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|
GTEX-111YS-2226-SM-5987P,0.0,0.0
GTEX-1122O-0005-SM-5O99J,21.523,0.0
GTEX-1H1ZS-0526-SM-9WG5L,8.922,0.0
GTEX-1122O-0326-SM-5H124,0.0,0.0
```

not clear if pandas will be able to tranform this file or not. It has been running for a couple of day
```
$ ll /scratch/aedavids/sortedGTExTrainSalmonNumReadsTransposeMatrix.csv 
-rw-r--r-- 1 aedavids prismuser 212G Jan 11 20:36 /scratch/aedavids/sortedGTExTrainSalmonNumReadsTransposeMatrix.csv
```

Can we use Spark?

- https://spark.apache.org/docs/latest/mllib-data-types.html#blockmatrix
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.linalg.distributed.BlockMatrix.html

In [1]:
SPARK_HOME="../deseq/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init( SPARK_HOME )

In [2]:
from pyspark.sql import SparkSession
from pyspark.mllib.linalg.distributed import BlockMatrix

In [3]:
spark = SparkSession\
                .builder\
                .appName("transpose")\
                .getOrCreate()

In [4]:
testMatrix = "../deseq/python/test/data/pythonTestSalmonNumReadsTransposeMatrix.csv"
df = spark.read.csv( testMatrix)
print( df.columns )
print()
df.show()

['_c0', '_c1', '_c2']

+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|
+--------------------+--------------------+--------------------+
|                Name|ENST00000456328.2...|ENST00000450305.2...|
|GTEX-111YS-2226-S...|                 0.0|                 0.0|
|GTEX-1122O-0005-S...|              21.523|                 0.0|
|GTEX-1H1ZS-0526-S...|               8.922|                 0.0|
|GTEX-1122O-0326-S...|                 0.0|                 0.0|
+--------------------+--------------------+--------------------+



In [5]:
# colnames do not show nicely
# todo add schema
df = spark.read.csv( testMatrix, header=True)
df.show()

+--------------------+----------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|                Name|ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript||ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene||
+--------------------+----------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|GTEX-111YS-2226-S...|                                                                                                                        

In [9]:
# https://stackoverflow.com/a/53149013/4586180
def fixColNames( df ) :
    '''
    Column headers: lower case + remove spaces and the following characters: ,;{}()=  
    '.' replaced with 'xxx'
    '''
    newColumns = []
    problematic_chars = ',;{}()='
    for columnStr in df.columns:
        columnStr = columnStr.lower()
        columnStr = columnStr.replace(' ', '_')
        columnStr = columnStr.replace('.', 'xxx')
        for c in problematic_chars:
            columnStr = columnStr.replace(c, '')
        newColumns.append(columnStr)
    df = df.toDF(*newColumns)
    
    return df

fixedDF = fixColNames( df )
print(" ")
print(fixedDF.columns)
print()

fixedDF.select( fixedDF.columns[1:]).show()

 
['name', 'enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-202|ddx11l1|1657|processed_transcript|', 'enst00000450305xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000002844xxx2|ddx11l1-201|ddx11l1|632|transcribed_unprocessed_pseudogene|']

+------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-202|ddx11l1|1657|processed_transcript||enst00000450305xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000002844xxx2|ddx11l1-201|ddx11l1|632|transcribed_unprocessed_pseudogene||
+------------------------------------------------------------------------------------------------------------------------------------+-------

In [11]:
realDF = fixedDF.select( fixedDF.columns[1:])

In [15]:
for row in realDF.rdd.collect() :
    print()
    print(row)
    print(row.)


Row(enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-202|ddx11l1|1657|processed_transcript|='0.0', enst00000450305xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000002844xxx2|ddx11l1-201|ddx11l1|632|transcribed_unprocessed_pseudogene|='0.0')

Row(enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-202|ddx11l1|1657|processed_transcript|='21.523', enst00000450305xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000002844xxx2|ddx11l1-201|ddx11l1|632|transcribed_unprocessed_pseudogene|='0.0')

Row(enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-202|ddx11l1|1657|processed_transcript|='8.922', enst00000450305xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000002844xxx2|ddx11l1-201|ddx11l1|632|transcribed_unprocessed_pseudogene|='0.0')

Row(enst00000456328xxx2|ensg00000223972xxx5|otthumg00000000961xxx2|otthumt00000362751xxx1|ddx11l1-20

In [10]:
rowsPerBlock = 1
colsPerBlock = 1
bmat1 = BlockMatrix(realDF.rdd, rowsPerBlock, colsPerBlock )
bmat1

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (192.168.68.110 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/andrewdavidson/googleUCSC/kimLab/extraCellularRNA/terra/deseq/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/Users/andrewdavidson/googleUCSC/kimLab/extraCellularRNA/terra/deseq/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/andrewdavidson/googleUCSC/kimLab/extraCellularRNA/terra/deseq/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "../des